# Gradient Descent: Part II

Chapter [3](03-gradientdescent.html) showed how to leverage gradient descent approach to find the minimal point for a given function. In this chapter, I will show how this idea relates to linear regression and other machine learning related algorithms. 

In the case of linear regression we have a given set of data point (X, y) and we want to identify the underlying function that generated these data points. On the surface this looks like a very different problem than finding the optimal point at which a function maximizes or minimizes and, therefore, hard to understand how gradient descent approach can be used over here. But let's rethink about the problem again in relation to chapter [1](01-regressionmetrics.html). 

In Chapter 1, I talked about how to evaluate models that generate continuous values. One of the metric we discussed was "Mean Absolute Error" or MAE. As the name suggests, it essentially the mean of the difference between actual ($y$) and predicted ($\hat{y}$) values and mathematically represented as 

$$MAE = \frac{\sum_{i=1}^{N}|y_i - \hat{y_i}|}{N}$$

Now, we have two options to find our best model. One is to have a two step process where somehow we come with several different functional form for our observed data points and calculate MAE for each of them and select the one with minimal MAE. The second option is to directly redefine the function as minimizing MAE i.e. the objective is to minimize MAE. However, it is still now clear what can we modify to minimize MAE as there is no given functional form of the equation. Well, this is not completely true though. 

Let's assume that data our observed data was generated using a function as given below

$y = 4 * x_0^2 + 2*x_1^2$

The below code generates some random data points using the above parabola function and plots it. 


In [5]:
from numpy.random import normal
import pandas as pd
import random

dataDF = pd.DataFrame({
  'x1':  [random.randint(-100, 100) for x in range(1000)],
  'x2':  [random.randint(-100, 100) for x in range(1000)]
})
dataDF['y'] = 4 * dataDF['x1']**2 + 2 * dataDF['x2']**2 

from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go
# init_notebook_mode(connected=False)
iplot([go.Scatter3d(x=dataDF['x1'], y=dataDF['x2'], z=dataDF['y'], opacity=0.5, mode='markers')], show_link=False)

In [3]:
from copy import copy
from scipy.spatial import distance
from random import random
from IPython.display import HTML
import numpy as np

def gradient(X, y, theta):
    num_samples = X.shape[0]
    y_prime = np.sum(np.transpose(theta)*X, axis=1)
    diff = y - y_prime
    gradient = -1. * np.sum(diff.reshape((num_samples, 1)) * X, axis=0) / float(num_samples)
    return gradient


def ols(X, y, eta = 0.03, fun=gradient, max_iterations = 1000, traceback = None, stopping_threshold = 1.0e-6):
    
    # number of samples
    m = X.shape[0]
    
    # number of parameters
    c = X.shape[1]
    
    # starting point -- randomly select
    theta1 = [random() for i in range(c)]
    
    for iter in range(max_iterations):
        
        if traceback is not None:
            traceback.append(copy(theta1))

        theta2  = theta1 - eta * gradient(X, y, theta1)
        
        # check if we reached stopping criteria threshold
        if distance.euclidean(theta2, theta1) < stopping_threshold:
            return theta1
        else:
            theta1 = theta2
        
    # if we reached max iterations then return current point
    return theta1



X = (dataDF[['x1', 'x2']] ** 2).values
y = dataDF['y'].values
traceback = []
theta = ols(X, y, eta=1e-8, max_iterations=10000, traceback=traceback)
display(HTML("<strong>Optimized Parameters: {}".format(theta)))